In [1]:
import pandas as pd
import numpy as np
import os
from constants import DATA_PATH

Read raw data file

In [2]:
data = pd.read_excel(os.path.join(DATA_PATH, 'data_initial.xlsx'), header=None, index_col=False)

Preproccesing data

In [4]:
date = data.iloc[1, 1:]

equity = data.iloc[::43, 0]
data['equity'] = equity.reindex_like(data.drop(equity.index), method='ffill')

data = data.loc[~(data.loc[:, 0] == 'Date')]
data = data.rename(columns={0: 'factor'})

data['equity'] = data['equity'].str.rsplit(" ", n=1, expand=True)[0]
data = data.set_index(['equity', 'factor'])
data.columns = date.values
data.columns.name = 'date'

data = data.loc[data.index.dropna()]
data = data.transpose().stack(0).swaplevel(0, 1).sort_index()
data.columns.name="factor"

# vado a settare le date uguali per ogni equity
data = data.reindex(pd.MultiIndex.from_product([data.index.get_level_values(0).unique(),
                                                data.index.get_level_values(1).unique()], names=data.index.names))

# vado a traslare le date in modo che i valori nulli siano all'inizio e non alla fine
data = data.groupby(level=0, group_keys=False).apply(lambda equity_df: equity_df.apply(
    lambda factor: np.roll(factor.values, factor.isna().sum())))

In [9]:
# data = data.drop(data.groupby(level=0).filter(lambda equity: (equity.isna().sum(axis=1) == len(equity.columns)).any()).index)
# data = data.drop(data.groupby(level=0).filter(lambda equity: equity['PX_LAST'].eq(0).any()).index).reset_index()

Save file preprocessed

In [10]:
data.to_csv('data/data.csv')